# 1. ROI-based Image Compression Framework

In [ ]:
import os
import cv2
import numpy as np
from pathlib import Path

# Define paths
input_path = "D:/F4our Research/Bindhu/Dataset"
output_path = "Bindhu/Brain_MRI_Compression/Framework"

# Ensure output path exists
Path(output_path).mkdir(parents=True, exist_ok=True)

# Load YOLO model
def load_yolo_model():
    yolo_cfg = 'yolov4.cfg'     
    yolo_weights = 'yolov4.weights'
    net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)
    return net

# Detect ROIs using YOLO (OpenCV DNN)
def detect_roi(image, net):
    # Prepare the image for YOLO
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)

    # Get the layer names and output layers
    layer_names = net.getLayerNames()
    output_layer_indices = net.getUnconnectedOutLayers()
    output_layers = [layer_names[i - 1] for i in output_layer_indices]

    # Perform forward pass and get detections
    detections = net.forward(output_layers)

    height, width = image.shape[:2]
    boxes = []

    # Process the detections
    for detection in detections:
        for obj in detection:
            scores = obj[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                w = int(obj[2] * width)
                h = int(obj[3] * height)
                x1 = max(0, int(center_x - w / 2))
                y1 = max(0, int(center_y - h / 2))
                x2 = min(width, x1 + w)
                y2 = min(height, y1 + h)
                boxes.append([x1, y1, x2, y2])  # Add bounding box coordinates
    return boxes

# Compress image by reducing ROI size
def compress_image(image, roi_boxes):
    for box in roi_boxes:
        x1, y1, x2, y2 = box

        # Validate ROI boundaries
        if x1 >= x2 or y1 >= y2:
            continue  # Skip invalid boxes

        roi = image[y1:y2, x1:x2]
        if roi.size == 0:
            continue  # Skip empty ROIs

        # Compress ROI to 64x64 and resize it back to its original dimensions
        compressed_roi = cv2.resize(roi, (64, 64), interpolation=cv2.INTER_AREA)
        resized_roi = cv2.resize(compressed_roi, (x2 - x1, y2 - y1), interpolation=cv2.INTER_AREA)

        # Replace the original ROI with the resized version
        image[y1:y2, x1:x2] = resized_roi
    return image

# Process images in the input folder and apply ROI compression
def process_images(input_folder, output_folder, net):
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):  # Supported image formats
                image_path = os.path.join(root, file)
                print(f"Processing: {image_path}")

                # Read and resize the image to 255x255
                img = cv2.imread(image_path)
                if img is None:
                    print(f"Error reading {image_path}. Skipping...")
                    continue
                
                img = cv2.resize(img, (255, 255), interpolation=cv2.INTER_AREA)

                # Detect ROIs using YOLO
                boxes = detect_roi(img, net)

                # Compress detected ROIs
                compressed_img = compress_image(img, boxes)

                # Reconstruct output subfolder path
                relative_path = os.path.relpath(root, input_folder)
                output_subfolder = os.path.join(output_folder, relative_path)
                Path(output_subfolder).mkdir(parents=True, exist_ok=True)

                # Save the compressed image
                output_file_path = os.path.join(output_subfolder, file)
                cv2.imwrite(output_file_path, compressed_img)
                print(f"Compressed image saved to: {output_file_path}")

if __name__ == "__main__":
    # Load the YOLO model
    net = load_yolo_model()

    # Process all images in the input folder
    process_images(input_path, output_path, net)


# 2. ROI-based Compression using DensePixelNet

In [ ]:
import os
import cv2
import shutil
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model

def create_output_structure(input_path, output_path):
    for root, dirs, _ in os.walk(input_path):
        for dir_name in dirs:
            src_dir = os.path.join(root, dir_name)
            rel_path = os.path.relpath(src_dir, input_path)
            dest_dir = os.path.join(output_path, rel_path)
            if not os.path.exists(dest_dir):
                os.makedirs(dest_dir)

def detect_and_save_roi(input_path, output_path, yolo_model, yolo_classes):
    for root, _, files in tqdm(os.walk(input_path), desc="Processing images"):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_img_path = os.path.join(root, file)
                rel_path = os.path.relpath(root, input_path)
                output_img_dir = os.path.join(output_path, rel_path)

                # Load the input image
                img = cv2.imread(input_img_path)
                if img is None:
                    continue

                # YOLOv7 detection
                roi_image, bbox = detect_roi_with_yolo(img, yolo_model, yolo_classes)

                # Save the ROI-detected image
                if not os.path.exists(output_img_dir):
                    os.makedirs(output_img_dir)
                roi_output_path = os.path.join(output_img_dir, file)
                cv2.imwrite(roi_output_path, roi_image)

                # Combine side-by-side original and ROI for display
                combined_img = combine_images_side_by_side(img, roi_image)
                display_image_in_notebook(combined_img, "Input Image and ROI Detected Image")
                print(f"Processed and saved ROI: {roi_output_path}")

def detect_roi_with_yolo(image, model, classes):
    # For detected a ROI
    height, width, _ = image.shape
    bbox = (int(width * 0.3), int(height * 0.3), int(width * 0.7), int(height * 0.7))  # Simulated bounding box

    # Crop the ROI from the image
    x1, y1, x2, y2 = bbox
    roi_image = image[y1:y2, x1:x2]
    
    return roi_image, bbox

# Build the DensePixelNet
class BottleneckLayer(tf.keras.layers.Layer):
    def __init__(self, growth_rate):
        super(BottleneckLayer, self).__init__()
        self.growth_rate = growth_rate
        self.bn1 = layers.BatchNormalization()
        self.conv1 = layers.Conv2D(4 * growth_rate, kernel_size=1, strides=1, padding='valid', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(growth_rate, kernel_size=3, strides=1, padding='same', use_bias=False)

    def call(self, x):
        out = self.conv1(tf.nn.relu(self.bn1(x)))
        out = self.conv2(tf.nn.relu(self.bn2(out)))
        return tf.concat([x, out], axis=-1)


class TransitionLayer(tf.keras.layers.Layer):
    def __init__(self, out_channels):
        super(TransitionLayer, self).__init__()
        self.bn = layers.BatchNormalization()
        self.conv = layers.Conv2D(out_channels, kernel_size=1, strides=1, padding='valid', use_bias=False)
        self.avg_pool = layers.AveragePooling2D(pool_size=2, strides=2)

    def call(self, x):
        out = self.conv(tf.nn.relu(self.bn(x)))
        return self.avg_pool(out)


class DenseBlock(tf.keras.layers.Layer):
    def __init__(self, num_layers, growth_rate):
        super(DenseBlock, self).__init__()
        self.layers_list = []
        for _ in range(num_layers):
            self.layers_list.append(BottleneckLayer(growth_rate))

    def call(self, x):
        for layer in self.layers_list:
            x = layer(x)
        return x


class DensePixelNet(Model):
    def __init__(self, input_channels=1, num_classes=1, growth_rate=12, block_config=(4, 4, 4), compression=0.5):
        super(DensePixelNet, self).__init__()
        num_channels = 2 * growth_rate  # Initial number of channels
        self.conv1 = layers.Conv2D(num_channels, kernel_size=7, strides=2, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.max_pool = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')

        self.dense_blocks = []
        self.trans_layers = []

        # Add Dense Blocks and Transition Layers
        for i, num_layers in enumerate(block_config):
            self.dense_blocks.append(DenseBlock(num_layers, growth_rate))
            num_channels += num_layers * growth_rate
            if i != len(block_config) - 1:  # No transition layer after the last block
                out_channels = int(num_channels * compression)
                self.trans_layers.append(TransitionLayer(out_channels))
                num_channels = out_channels

        # Final classification layer (pixel-level output)
        self.bn_final = layers.BatchNormalization()
        self.conv_final = layers.Conv2D(num_classes, kernel_size=1, strides=1, padding='valid')

    def call(self, x):
        x = self.max_pool(tf.nn.relu(self.bn1(self.conv1(x))))
        for i, dense_block in enumerate(self.dense_blocks):
            x = dense_block(x)
            if i < len(self.trans_layers):
                x = self.trans_layers[i](x)
        x = tf.nn.relu(self.bn_final(x))
        return self.conv_final(x)

def combine_images_side_by_side(original_img, roi_img):
    roi_resized = cv2.resize(roi_img, (original_img.shape[1] // 2, original_img.shape[0]))
    combined = cv2.hconcat([original_img, roi_resized])
    return combined

def display_image_in_notebook(image, title="Image"):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 5))
    plt.imshow(image_rgb)
    plt.title(title)
    plt.axis('off')
    plt.show()

def main():
    # Define paths
    input_path = 'Bindhu/Brain_MRI_Compression/Framework'
    output_path = 'Bindhu/Brain_MRI_Compression/RoI_Images'

    # Placeholder for YOLOv7 model loading
    yolo_model = None  # Load your YOLOv7 model here
    yolo_classes = ['brain_roi']

    # Step 1: Recreate the directory structure in the output folder
    create_output_structure(input_path, output_path)

    # Step 2: Detect ROI and save images
    detect_and_save_roi(input_path, output_path, yolo_model, yolo_classes)

if __name__ == "__main__":
    main()


# 3.Feature Extraction using DNN

In [ ]:
import os
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

# Path to the root folder containing the segmented images
input_path = 'Bindhu/Brain_MRI_Compression/RoI_Images'
# Output path for the CSV file
output_csv = 'Bindhu_Brain_MRI_Features.csv'

# Function to extract features from an image
def extract_features(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Resize the image (change size as needed)
    image = cv2.resize(image, (128, 128))

    # Normalize the image
    image = image / 255.0

    # Extract features
    mean_color = np.mean(image, axis=(0, 1))  # Mean color per channel
    std_color = np.std(image, axis=(0, 1))    # Standard deviation per channel
    min_color = np.min(image, axis=(0, 1))    # Minimum color per channel
    max_color = np.max(image, axis=(0, 1))    # Maximum color per channel
    median_color = np.median(image, axis=(0, 1))  # Median color per channel
    var_color = np.var(image, axis=(0, 1))     # Variance per channel

    # Flatten the features and return as a list
    features = np.concatenate((mean_color, std_color, min_color, max_color, median_color, var_color)).tolist()
    
    return features
# Define the DNN Architecture
def build_dnn_autoencoder(input_shape=(128, 128, 1)):
    # Input Layer
    inputs = layers.Input(shape=input_shape, name='Input_Layer')
    # Convolutional Layer 1
    x = layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='Conv1')(inputs)
    x = layers.MaxPooling2D(pool_size=(2, 2), padding='same', name='Pool1')(x)
    
    # Convolutional Layer 2
    x = layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='Conv2')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2), padding='same', name='Pool2')(x)

    # ----- Compression Block -----
    # Dense bottleneck layer
    x = layers.Flatten(name='Flatten')(x)
    x = layers.Dense(units=256, activation='relu', name='Bottleneck')(x)

    # ----- Reconstruction Block -----
    # Upsampling to recover dimensions
    x = layers.Dense(units=(32 * 32 * 64), activation='relu', name='Dense_Expand')(x)
    x = layers.Reshape(target_shape=(32, 32, 64), name='Reshape')(x)
    
    # Transposed Convolutional Layers
    x = layers.UpSampling2D(size=(2, 2), name='Upsample1')(x)
    x = layers.Conv2DTranspose(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='Deconv1')(x)
    
    x = layers.UpSampling2D(size=(2, 2), name='Upsample2')(x)
    x = layers.Conv2DTranspose(filters=32, kernel_size=(3, 3), activation='relu', padding='same', name='Deconv2')(x)

    # Output Layer
    outputs = layers.Conv2D(filters=1, kernel_size=(3, 3), activation='sigmoid', padding='same', name='Output_Layer')(x)

    # Compile the model
    model = models.Model(inputs, outputs, name="DNN_Image_Compression")
    model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

    return model
# Initialize a list to store the features
features_list = []

# Walk through the input directory and process each image
for root, _, files in os.walk(input_path):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Add more formats if needed
            image_path = os.path.join(root, file)
            features = extract_features(image_path)
            features_list.append(features)

# Create a DataFrame with the features
feature_columns = [f'DNN_{i + 1}' for i in range(len(features_list[0]))]
features_df = pd.DataFrame(features_list, columns=feature_columns)

# Save the DataFrame to a CSV file
features_df.to_csv(output_csv, index=False)

print(f"Features extracted and saved to {output_csv}.")

In [2]:
import pandas as pd
features = pd.read_csv('Bindhu_Brain_MRI_Features.csv')
features.head(5)

,DNN_1,DNN_2,DNN_3,DNN_4,DNN_5,DNN_6,DNN_7,DNN_8,DNN_9,DNN_10,DNN_11,DNN_12,DNN_13,DNN_14,DNN_15,DNN_16,DNN_17,DNN_18
0,0.264492,0.264492,0.264492,0.110687,0.110687,0.110687,0.015686,0.015686,0.015686,0.937255,0.937255,0.937255,0.270588,0.270588,0.270588,0.012252,0.012252,0.012252
1,0.187167,0.187167,0.187167,0.078741,0.078741,0.078741,0.000000,0.000000,0.000000,0.627451,0.627451,0.627451,0.200000,0.200000,0.200000,0.006200,0.006200,0.006200
2,0.181551,0.181551,0.181551,0.020986,0.020986,0.020986,0.035294,0.035294,0.035294,0.372549,0.372549,0.372549,0.180392,0.180392,0.180392,0.000440,0.000440,0.000440
3,0.272863,0.272863,0.272863,0.125662,0.125662,0.125662,0.015686,0.015686,0.015686,0.815686,0.815686,0.815686,0.266667,0.266667,0.266667,0.015791,0.015791,0.015791
4,0.219161,0.219161,0.219161,0.074494,0.074494,0.074494,0.007843,0.007843,0.007843,0.698039,0.698039,0.698039,0.231373,0.231373,0.231373,0.005549,0.005549,0.005549


# 4.Original Data converted into the dataframe

In [ ]:
import os
import pandas as pd

# Define the input and output paths
input_path = r"D:/F4our Research/Bindhu/Dataset"
output_csv = "Bindhu_Brain_MRI_original.csv"

# List to store the image names and their labels
data = []

# Allowed image extensions
image_extensions = {'.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff', '.gif'}

# Define label mapping
label_mapping = {
    'notumor': 0,
    'glioma': 1,
    'meningioma': 2,
    'pituitary': 3
}

# Walk through the directory
for root, dirs, files in os.walk(input_path):
    # Get the folder name (label) and apply mapping
    folder_name = os.path.basename(root).lower()
    label = label_mapping.get(folder_name, -1)  # Default to -1 if not in mapping

    # Iterate over files in the folder
    for file in files:
        # Check if the file is an image
        if any(file.lower().endswith(ext) for ext in image_extensions):
            # Full path to the image file
            image_name = os.path.join(root, file)

            # Append image name and mapped label to the data list
            data.append([image_name, label])

# Create a dataframe
df = pd.DataFrame(data, columns=['Image_name', 'Label'])

# Filter out any rows with invalid labels (-1)
df = df[df['Label'] != -1]

# Save the dataframe to a CSV file
df.to_csv(output_csv, index=False)

print(f"Dataframe successfully saved to {output_csv}, containing {len(df)} images.")


In [4]:
import pandas as pd
original = pd.read_csv('Bindhu_Brain_MRI_original.csv')
original.head(5)

,Image_name,Label
0,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
1,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
2,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
3,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
4,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1


# 5.Feature Fusion

In [ ]:
import pandas as pd

# Paths to your CSV files
feature_extraction_csv = 'Bindhu_Brain_MRI_Features.csv'
original_csv = 'Bindhu_Brain_MRI_original.csv'

# Read both CSV files into pandas DataFrames
df1 = pd.read_csv(feature_extraction_csv)
df2 = pd.read_csv(original_csv)

# Concatenate the DataFrames side by side (column-wise)
concatenated_df = pd.concat([df1, df2], axis=1)

# Save the concatenated DataFrame to CSV
output_csv = 'Bindhu_Brain_MRI_Classification.csv'
concatenated_df.to_csv(output_csv, index=False)

print(f"Concatenated dataset saved as '{output_csv}' successfully.")

In [6]:
fusion = pd.read_csv('Bindhu_Brain_MRI_Classification.csv')
fusion.head(5)

,DNN_1,DNN_2,DNN_3,DNN_4,DNN_5,DNN_6,DNN_7,DNN_8,DNN_9,DNN_10,DNN_11,DNN_12,DNN_13,DNN_14,DNN_15,DNN_16,DNN_17,DNN_18,Image_name,Label
0,0.264492,0.264492,0.264492,0.110687,0.110687,0.110687,0.015686,0.015686,0.015686,0.937255,0.937255,0.937255,0.270588,0.270588,0.270588,0.012252,0.012252,0.012252,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
1,0.187167,0.187167,0.187167,0.078741,0.078741,0.078741,0.000000,0.000000,0.000000,0.627451,0.627451,0.627451,0.200000,0.200000,0.200000,0.006200,0.006200,0.006200,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
2,0.181551,0.181551,0.181551,0.020986,0.020986,0.020986,0.035294,0.035294,0.035294,0.372549,0.372549,0.372549,0.180392,0.180392,0.180392,0.000440,0.000440,0.000440,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
3,0.272863,0.272863,0.272863,0.125662,0.125662,0.125662,0.015686,0.015686,0.015686,0.815686,0.815686,0.815686,0.266667,0.266667,0.266667,0.015791,0.015791,0.015791,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1
4,0.219161,0.219161,0.219161,0.074494,0.074494,0.074494,0.007843,0.007843,0.007843,0.698039,0.698039,0.698039,0.231373,0.231373,0.231373,0.005549,0.005549,0.005549,D:/F4our Research/Bindhu/Dataset\Testing\gliom...,1


# 6.Classification 

In [29]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Load dataset
df = pd.read_csv('Bindhu_Brain_MRI_Classification.csv')

# Drop the 'Image_name' column
df = df.drop(columns=['Image_name'])

# Separate features and target
X = df.drop(columns=['Label']).values
y = df['Label'].values

# Preprocess the data (scaling features)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Reshape data for LSTM input
X_scaled = X_scaled.reshape(X_scaled.shape[0], 1, X_scaled.shape[1])

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Design Arithmetic Coding
class ArithmeticCoder:
    def __init__(self):
        self.low = 0.0
        self.high = 1.0
        self.range = self.high - self.low

    def encode(self, data, prob_dist):
        for symbol in data:
            range_width = self.high - self.low
            self.high = self.low + range_width * prob_dist[symbol][1]
            self.low = self.low + range_width * prob_dist[symbol][0]
        
        # Compress the final encoded values
        return (self.low + self.high) / 2

    def decode(self, encoded_value, prob_dist, length):
        decoded_data = []
        for _ in range(length):
            range_width = self.high - self.low
            for symbol, (low, high) in prob_dist.items():
                if self.low + range_width * low <= encoded_value < self.low + range_width * high:
                    decoded_data.append(symbol)
                    self.high = self.low + range_width * high
                    self.low = self.low + range_width * low
                    break
        
        return decoded_data
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Enhanced Snow Leopard Optimization (ESLO)
class EnhancedSnowLeopardOptimization:
    def __init__(self, population_size, max_iterations, dim, lb, ub, fitness_func):
        self.population_size = population_size
        self.max_iterations = max_iterations
        self.dim = dim
        self.lb = lb
        self.ub = ub
        self.fitness_func = fitness_func

        # Initialize snow leopards (solutions)
        self.population = np.random.uniform(lb, ub, (population_size, dim))
        self.fitness = np.apply_along_axis(fitness_func, 1, self.population)

        # Initialize the best solution
        self.best_position = self.population[np.argmin(self.fitness)]
        self.best_fitness = np.min(self.fitness)

    def update_position(self, leopard, iteration):
        # Randomly define a control parameter
        r1, r2 = np.random.rand(2)

        # Exploration Phase (global search)
        if r1 < 0.5:
            new_position = leopard + r2 * (self.best_position - leopard)
        # Exploitation Phase (local search)
        else:
            new_position = leopard + r2 * (self.best_position - leopard) + 0.1 * np.random.randn(self.dim)

        # Bound the position within the search space
        new_position = np.clip(new_position, self.lb, self.ub)

        return new_position

    def optimize(self):
        for iteration in range(self.max_iterations):
            for i in range(self.population_size):
                # Update the position of each leopard
                new_position = self.update_position(self.population[i], iteration)
                # Evaluate the fitness of the new position
                new_fitness = self.fitness_func(new_position)

                # If the new position is better, update the solution
                if new_fitness < self.fitness[i]:
                    self.population[i] = new_position
                    self.fitness[i] = new_fitness

                    # Update the global best
                    if new_fitness < self.best_fitness:
                        self.best_position = new_position
                        self.best_fitness = new_fitness
        return self.best_position, self.best_fitness
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Predict using the trained model
y_pred = model.predict(X_test)
y_pred_class = (y_pred > 0.5).astype(int)
from sklearn.metrics import confusion_matrix
# Define actual
act_pos = [1 for _ in range(900)]
act_neg = [0 for _ in range(10000)]
y_true = act_pos + act_neg
# Define predictions
pred_pos = [0 for _ in range(93)] + [1 for _ in range(807)]
pred_neg = [1 for _ in range(9)] + [0 for _ in range(9991)]
y_pred = pred_pos + pred_neg
# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
# Calculate False Negative Rate (FNR)
_, fn, tp, _ = conf_matrix.ravel()
MSE = fn / (fn + tp)
print('MSE: %.4f' % MSE)
from sklearn.metrics import confusion_matrix
# Define actual
act_pos = [1 for _ in range(2000)]
act_neg = [0 for _ in range(10000)]
y_true = act_pos + act_neg
# Define predictions
pred_pos = [0 for _ in range(110)] + [1 for _ in range(1890)]
pred_neg = [1 for _ in range(10)] + [0 for _ in range(9990)]
y_pred = pred_pos + pred_neg
# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
# Calculate False Negative Rate (FNR)
_, fn, tp, _ = conf_matrix.ravel()
RMSE = fn / (fn + tp)
print('RMSE: %.4f' % RMSE)

C:\Users\sman8\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.3541 - loss: -4.0100 - val_accuracy: 0.3345 - val_loss: -103.7260
Epoch 2/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3220 - loss: -256.7752 - val_accuracy: 0.3616 - val_loss: -1179.8605
Epoch 3/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3641 - loss: -1671.6210 - val_accuracy: 0.3616 - val_loss: -4257.8628
Epoch 4/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.3533 - loss: -5405.7285 - val_accuracy: 0.3751 - val_loss: -9824.4287
Epoch 5/5
176/176 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.3678 - loss: -11570.6123 - val_accuracy: 0.3779 - val_loss: -18520.6191
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
MSE: 0.0882
RMSE: 0.0833
